In [1]:
from ultralytics import YOLO
import numpy as np
model = YOLO("yolov8n-pose.pt",task="pose")

WARNING ⚠️ user config directory is not writeable, defaulting to '\tmp\Ultralytics'.


In [2]:
SOL_OMUZ = 5
SAG_OMUZ = 6
SAG_BILEK = 16
SOL_BILEK = 15
SOL_BEL = 11
SAG_BEL = 12
SOL_DIRSEK = 7
SAG_DIRSEK = 8
SOL_EL = 9
SAG_EL = 10
SOL_DIZ = 13
SAG_DIZ = 14

In [3]:
import math
def comelme_hesapla(sol_bilek,sol_diz,sol_bel):
    p1 = math.dist(sol_bilek,sol_diz)
    p2 = math.dist(sol_diz,sol_bel)
    p3 = math.dist(sol_bel,sol_bilek)
    aci = math.degrees(math.acos((p1*p1+p2*p2-p3*p3)/(2*p1*p2)))
    if aci<90:
        return True
    else:
        return False

    

In [4]:
def yatma_hesapla(sol_dirsek,sol_omuz,sol_bel):
    p1 = math.dist(sol_dirsek,sol_omuz)
    p2 = math.dist(sol_omuz,sol_bel)
    p3 = math.dist(sol_bel,sol_dirsek)
    aci = math.degrees(math.acos((p1*p1+p2*p2-p3*p3)/(2*p1*p2)))
    if aci>30:
        return True
    else:
        return False


In [5]:
def kosma_hesapla(sol_el,sol_dirsek,sol_omuz):
    p1 = math.dist(sol_el,sol_dirsek)
    p2 = math.dist(sol_dirsek,sol_omuz)
    p3 = math.dist(sol_omuz,sol_el)
    aci = math.degrees(math.acos((p1*p1+p2*p2-p3*p3)/(2*p1*p2)))
    if aci<90:
        return True
    else:
        return False

In [14]:
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO

# Load the YOLOv8 model

# Open the video file
video_path = r"fainting_videos\Denmark official faints during Covid-19 conference.mp4"
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
writer = cv2.VideoWriter("test1.mp4",fourcc,30,(int(cap.get(3)),int(cap.get(4))),True)
# Store the track history
track_history = defaultdict(lambda: [])
frame_count = 0
frame_count_last = {}
frame_count_first = {}

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        try:    
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True)

        # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            keypoints = results[0].keypoints.xy.cpu()
            
        # Visualize the results on the frame
            annotated_frame = results[0].plot()
            if len(track_ids):
                for i,trk_id in enumerate(track_ids):
                    if trk_id not in frame_count_first:
                        frame_count_last[trk_id] = frame_count
                        frame_count_first[trk_id] = frame_count
                    if frame_count_last[trk_id]>frame_count_first[trk_id]:
                        frame_count_last[trk_id] = frame_count
                    else:
                        frame_count_first[trk_id] = frame_count
        # Plot the tracks
            for box, track_id,keypoint in zip(boxes, track_ids,keypoints):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                #if len(track) > 30:  # retain 90 tracks for 90 frames
                #    track.pop(0)
                sol_bel = keypoint.xy.cpu()[0][SOL_BEL]
                sol_diz = keypoint.xy.cpu()[0][SOL_DIZ]
                sol_bilek = keypoint.xy.cpu()[0][SOL_BILEK]
                sol_el = keypoint.xy.cpu()[0][SOL_EL]
                sol_dirsek= keypoint.xy.cpu()[0][SOL_DIRSEK]
                sol_omuz = keypoint.xy.cpu()[0][SOL_OMUZ]
                if comelme_hesapla(sol_bilek=sol_bilek,sol_diz=sol_diz,sol_bel=sol_bel):
                    cv2.putText(annotated_frame,"comelme",(int(x),int(y+20)),cv2.FONT_HERSHEY_COMPLEX,1,(230,230,230),1)
                elif kosma_hesapla(sol_el=sol_el,sol_dirsek=sol_dirsek,sol_omuz=sol_omuz):
                    cv2.putText(annotated_frame,"kosma",(int(x),int(y+20)),cv2.FONT_HERSHEY_COMPLEX,1,(230,230,230),1)
                elif yatma_hesapla(sol_dirsek=sol_dirsek,sol_omuz=sol_omuz,sol_bel=sol_bel):
                    cv2.putText(annotated_frame,"yatma",(int(x),int(y+20)),cv2.FONT_HERSHEY_COMPLEX,1,(230,230,230),1)
                else:
                    cv2.putText(annotated_frame,"hicbirsey yapmiyor.",(int(x),int(y+20)),cv2.FONT_HERSHEY_COMPLEX,1,(230,230,230),1)
            # Draw the tracking lines
                duration = (frame_count_first[track_id]-frame_count_last[track_id])/30
                cv2.putText(annotated_frame,f"duration: {round(duration)}",(int(x),int(y+10)),cv2.FONT_HERSHEY_COMPLEX,1,(230,230,230),1)

        # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
            writer.write(annotated_frame)
            
        # Break the loop if 'q' is pressed
            if cv2.waitKey(30) & 0xFF == ord("q"):
                break
        except (AttributeError):
            continue
        frame_count+=1
    else:
        # Break the loop if the end of the video is reached
        break
    

# Release the video capture object and close the display window
cap.release()
writer.release()
cv2.destroyAllWindows()

0: 640x640 2 persons, 128.8ms
Speed: 6.0ms preprocess, 128.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 15.4ms
Speed: 0.0ms preprocess, 15.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 6.7ms
Speed: 6.1ms preprocess, 6.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



[355, 356]
[355, 356]
[355, 356]


0: 640x640 2 persons, 13.3ms
Speed: 0.0ms preprocess, 13.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 10.0ms
Speed: 4.4ms preprocess, 10.0ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 11.1ms
Speed: 2.0ms preprocess, 11.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 7.8ms
Speed: 7.2ms preprocess, 7.8ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 640)



[355, 356]
[355, 356]
[355, 356]
[355, 356]


0: 640x640 2 persons, 8.3ms
Speed: 7.5ms preprocess, 8.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 4.8ms
Speed: 3.2ms preprocess, 4.8ms inference, 6.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 11.8ms
Speed: 2.7ms preprocess, 11.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 12.2ms
Speed: 0.0ms preprocess, 12.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 7.7ms


...

Speed: 1.5ms preprocess, 12.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 9.9ms
Speed: 3.0ms preprocess, 9.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 7.7ms
Speed: 3.0ms preprocess, 7.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 10.5ms
Speed: 4.0ms preprocess, 10.5ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 5.7ms
Speed: 6.6ms preprocess, 5.7ms inference, 6.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.6ms
Speed: 4.0ms preprocess, 9.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.4ms


[413]
[413]
[413]
[413]


Speed: 3.0ms preprocess, 10.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.0ms
Speed: 5.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.8ms
Speed: 7.1ms preprocess, 10.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.0ms
Speed: 3.9ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.7ms
Speed: 7.0ms preprocess, 9.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.4ms
Speed: 3.5ms preprocess, 9.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 12.8ms
Speed: 0.0ms preprocess, 12.8ms inference, 2.7ms postprocess per image at shape (1

[413]
[413]
[413]
[413]
[413]


0: 640x640 (no detections), 8.9ms
Speed: 6.6ms preprocess, 8.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.9ms
Speed: 3.0ms preprocess, 8.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.0ms
Speed: 4.0ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.0ms
Speed: 3.0ms preprocess, 10.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.0ms
Speed: 7.6ms preprocess, 8.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.9ms
Speed: 6.1ms preprocess, 8.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.9ms
Speed: 3.0ms preprocess, 9.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.1ms
Speed: 3.0ms preprocess, 12.1ms inference, 0